In [1]:
import torch
import requests
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

In [2]:
%matplotlib notebook
%matplotlib inline

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
A_OK_PATH='/Data_Storage/Rui_Data_Space/VQA/A-OKVQA'
PATH='/Data_Storage/Rui_Data_Space/VQA'
import pandas as pd
from collections import defaultdict

In [3]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [4]:
import os
import random

In [5]:
from sklearn.cluster import KMeans

In [6]:
from transformers import BertModel, AutoTokenizer
model_name='bert-base-uncased'
bert=BertModel.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
import openai
#engine_name='code-davinci-002'
engine_name='text-davinci-003'
temperature=0.5
max_tokens=128
top_p=1
frequency_penalty=0.0
presence_penalty=0.0
num_sequence=1
openai.api_key =open('OK_VQA/openai_key.txt','r').readlines()[0].strip()

In [5]:
train_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_train.json'))
val_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_val.json'))
test_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_test.json'))
print(len(train_aokvqa),len(val_aokvqa),len(test_aokvqa))

17056 1145 6702


In [6]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [ ]:
prompt_head=open('OK_VQA/demonstrations.txt','r').readlines()
head=''.join(prompt_head)

In [10]:
import time

In [23]:
print (val_aokvqa[0])

{'split': 'val', 'image_id': 461751, 'question_id': '22jbM6gDxdaMaunuzgrsBB', 'question': "What is in the motorcyclist's mouth?", 'choices': ['toothpick', 'food', 'popsicle stick', 'cigarette'], 'correct_choice_idx': 3, 'direct_answers': ['cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette', 'cigarette'], 'difficult_direct_answer': False, 'rationales': ["He's smoking while riding.", 'The motorcyclist has a lit cigarette in his mouth while he rides on the street.', 'The man is smoking.']}


In [8]:
captions=load_pkl('EACL-24-results/aokvqa_val_captions_100.pkl')
print (len(captions))

1145


In [18]:
kb_info=load_pkl('EACL-24-results/aokvqa_val_kb.pkl')
print(len(kb_info))

1145


In [21]:
import time

In [ ]:
NUM_VIS=10
NUM_CAP=5

vis=0
total={}
invalid=[]
for k,row in enumerate(val_aokvqa):
    if k%100==0:
        print ('Already finished:',k,k*100.0/len(val_aokvqa))
    ques=row['question']
    ques_id=str(row['question_id'])
    
    cap=captions[ques_id]
    
    context=' . '.join(cap[:NUM_CAP])
    texts=[]
    texts.append('Context:'+context)
    texts.append('Question:'+ques)
    texts.append('Knowledge:')
    texts='\n'.join(texts)
    prompt=head+texts
    
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=prompt,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            top_p=top_p,
                                            frequency_penalty=frequency_penalty,
                                            presence_penalty=presence_penalty,
                                            stop=["\n"])
        knowledge=response["choices"][0]["text"].strip()
        total[ques_id]=knowledge
        time.sleep(0.5)
        vis+=1
    except:
        invalid.append(ques_id)
        vis+=1
        print ('Invalid id:',ques_id)
        continue


In [17]:
pkl.dump(total,open('EACL-24-results/aokvqa_val_kb.pkl','wb'))

In [12]:
total=load_pkl('EACL-24-results/aokvqa_val_kb.pkl')
print(len(total))

1128


In [15]:
NUM_VIS=10
NUM_CAP=5

vis=0
invalid=[]
for k,row in enumerate(val_aokvqa):
    if k%100==0:
        print ('Already finished:',k,k*100.0/len(val_aokvqa))
    ques=row['question']
    ques_id=str(row['question_id'])
    if ques_id in total:
        continue
    cap=captions[ques_id]
    
    context=' . '.join(cap[:NUM_CAP])
    texts=[]
    texts.append('Context:'+context)
    texts.append('Question:'+ques)
    texts.append('Knowledge:')
    texts='\n'.join(texts)
    prompt=head+texts
    
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=prompt,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            top_p=top_p,
                                            frequency_penalty=frequency_penalty,
                                            presence_penalty=presence_penalty,
                                            stop=["\n"])
        knowledge=response["choices"][0]["text"].strip()
        total[ques_id]=knowledge
        time.sleep(0.5)
        vis+=1
    except:
        invalid.append(ques_id)
        vis+=1
        print ('Invalid id:',ques_id)
        continue


Already finished: 0 0.0
Already finished: 100 8.733624454148472
Already finished: 200 17.467248908296945
Already finished: 300 26.200873362445414
Already finished: 400 34.93449781659389
Already finished: 500 43.66812227074236
Already finished: 600 52.40174672489083
Already finished: 700 61.1353711790393
Already finished: 800 69.86899563318778
Already finished: 900 78.60262008733625
Already finished: 1000 87.33624454148472
Already finished: 1100 96.06986899563319


In [16]:
print (len(total))

1145


In [13]:
pkl.dump(total,open('OK_VQA/aokvqa_demo_kb.pkl','wb'))